In [1]:
import pandas as pd
import random
from datetime import datetime, timedelta

# Load datasets
mentors = pd.read_csv("Mentor.csv")  # Columns: Name, Sector 1, Sector 2, Sector 3, Index
startups = pd.read_csv("Startups.csv")  # Columns: Name, Sector, Index

# Initialize time slots
start_time = datetime.strptime("11:00 AM", "%I:%M %p")
end_time = datetime.strptime("2:00 PM", "%I:%M %p")
time_slots = []
slot_duration = timedelta(minutes=15)
gap_duration = timedelta(minutes=5)

while start_time + slot_duration <= end_time:
    time_slots.append(start_time.strftime("%I:%M %p"))
    start_time += slot_duration + gap_duration

# Helper function to assign startups to mentors based on preferences
def assign_startups(mentors, startups, time_slots):
    mentor_schedule = {mentor["Name"]: [] for _, mentor in mentors.iterrows()}
    startup_counts = {startup["Name"]: 0 for _, startup in startups.iterrows()}
    assigned_startups = set()
    
    for _, mentor in mentors.iterrows():
        preferences = [mentor["Sector 1"], mentor["Sector 2"], mentor["Sector 3"]]
        weights = [60, 24, 16]
        mentor_startups = []

        # Assign startups based on sector preference
        for _ in range(len(time_slots)):
            # Stop assigning if all time slots for the mentor are used
            if len(mentor_startups) >= len(time_slots):
                break

            available_startups = startups[
                ~startups["Name"].isin([s["Name"] for s in mentor_startups]) &
                startups["Sector"].isin(preferences)
            ]

            if available_startups.empty:
                continue

            # Select a startup based on weighted preferences
            chosen_sector = random.choices(preferences, weights=weights, k=1)[0]
            sector_startups = available_startups[available_startups["Sector"] == chosen_sector]
            
            if sector_startups.empty:
                continue
            
            startup = sector_startups.sample(1).iloc[0]
            mentor_startups.append({"Name": startup["Name"], "Sector": startup["Sector"]})
            assigned_startups.add(startup["Name"])
            startup_counts[startup["Name"]] += 1

            # Stop assigning if startup count exceeds limit
            if startup_counts[startup["Name"]] >= 4:
                break

        mentor_schedule[mentor["Name"]] = mentor_startups

    # Reallocate remaining startups
    for _, startup in startups.iterrows():
        if startup["Name"] not in assigned_startups:
            for mentor in mentors["Name"]:
                if startup_counts[startup["Name"]] < 4:
                    # Ensure the mentor still has available time slots
                    if len(mentor_schedule[mentor]) < len(time_slots):
                        mentor_schedule[mentor].append({"Name": startup["Name"], "Sector": startup["Sector"]})
                        assigned_startups.add(startup["Name"])
                        startup_counts[startup["Name"]] += 1
                        break

    return mentor_schedule, startup_counts

# Assign startups to mentors
schedule, startup_counts = assign_startups(mentors, startups, time_slots)

# Assign time slots to mentors
final_schedule = []
for mentor, startups in schedule.items():
    slots = iter(time_slots)
    for startup in startups:
        final_schedule.append({
            "Mentor": mentor,
            "Startup": startup["Name"],
            "Sector": startup["Sector"],
            "Time Slot": next(slots, "N/A")
        })

# Display the schedule
print("Mentoring Schedule:")
for _, mentor in mentors.iterrows():
    print(f"\nMentor: {mentor['Name']}")
    print(f"Preferences: {mentor['Sector 1']}, {mentor['Sector 2']}, {mentor['Sector 3']}")
    print("Scheduled Startups:")
    for session in [s for s in final_schedule if s["Mentor"] == mentor["Name"]]:
        print(f"  - {session['Startup']} ({session['Sector']}) at {session['Time Slot']}")

# Display startup counts
print("\nStartup Session Counts:")
for startup, count in startup_counts.items():
    print(f"{startup}: {count} sessions")


Mentoring Schedule:

Mentor: Mentor 1
Preferences: B2B Software, Marketplace, Media/Media Tech
Scheduled Startups:
  - Startup 91 (Media/Media Tech) at 11:00 AM
  - Startup 69 (Media/Media Tech) at 11:20 AM
  - Startup 60 (B2B Software) at 11:40 AM
  - Startup 70 (B2B Software) at 12:00 PM
  - Startup 78 (B2B Software) at 12:20 PM
  - Startup 44 (B2B Software) at 12:40 PM
  - Startup 128 (B2B Software) at 01:00 PM
  - Startup 123 (B2B Software) at 01:20 PM
  - Startup 23 (Marketplace) at 01:40 PM

Mentor: Mentor 2
Preferences: Consumer goods and services, HR Tec, Government
Scheduled Startups:
  - Startup 49 (HR Tec) at 11:00 AM
  - Startup 6 (Consumer goods and services) at 11:20 AM
  - Startup 130 (HR Tec) at 11:40 AM
  - Startup 40 (Consumer goods and services) at 12:00 PM
  - Startup 133 (Consumer goods and services) at 12:20 PM
  - Startup 82 (Consumer goods and services) at 12:40 PM
  - Startup 74 (Consumer goods and services) at 01:00 PM
  - Startup 146 (Consumer goods and servi